In [1]:
import numpy as np
import os
import pandas as pd
from server import Server 

from client import Client

In [2]:
#import pyximport
#pyximport.install(setup_args={"script_args" : ["--verbose"]})
#from linbinR import fast_linbin

In [3]:
split_dir = "/home/silke/Documents/Fed_EWAS/Data/QC_GSE134379_half/GSE134379_splits"
output = "/home/silke/Documents/Fed_EWAS/Data/QC_GSE134379_half/GSE134379_Fed"
probeAnnotationPath = "/home/silke/Documents/Fed_EWAS/Data/GSE134379_RAW/GPL13534_HumanMethylation450_15017482_v.1.1.csv"

## Initialising the clients

In [4]:
# create client
lab_a = Client("Lab_A", os.path.join(split_dir, "Split_1_design.csv"), os.path.join(split_dir, "Split_1_methylated.csv"), os.path.join(split_dir, "Split_1_unmethylated.csv"), probeAnnotationPath)
lab_b = Client("Lab_B", os.path.join(split_dir, "Split_2_design.csv"), os.path.join(split_dir, "Split_2_methylated.csv"), os.path.join(split_dir, "Split_2_unmethylated.csv"), probeAnnotationPath)
lab_c = Client("Lab_C", os.path.join(split_dir, "Split_3_design.csv"), os.path.join(split_dir, "Split_3_methylated.csv"), os.path.join(split_dir, "Split_3_unmethylated.csv"), probeAnnotationPath)

/home/silke/Documents/Fed_EWAS/Federated_Differential_Methylation_Analysis/Federated_Implementation/client.py:81: DtypeWarning: Columns (2,4,11,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  self.read_data(design_matrix_filepath, methylated_filepath, unmethylated_filepath, probe_annotation_path)


## Initialising the server

In [5]:
serv = Server(["AD", "CTRL"], ["Age", "Sex", "Sentrix_ID"])
global_conditions = serv.return_global_conditions()

## Joining clients to the server

In [6]:
# join the clients
serv.get_clients(lab_a.cohort_name, lab_a.probes, lab_a.designmatrix.index)
serv.get_clients(lab_b.cohort_name, lab_b.probes, lab_b.designmatrix.index)
serv.get_clients(lab_c.cohort_name, lab_c.probes, lab_c.designmatrix.index)

In [7]:
global_probes = serv.find_global_probes()
cohort_effect = serv.find_cohort_effects()

In [8]:
#check client input
lab_a.input_validation(global_conditions, global_probes)
lab_b.input_validation(global_conditions, global_probes)
lab_c.input_validation(global_conditions, global_probes)

In [9]:
lab_a.cohort_effects(serv.cohort_effects)
lab_b.cohort_effects(serv.cohort_effects)
lab_c.cohort_effects(serv.cohort_effects)

In [10]:
if "Sentrix_ID" in global_conditions:
    lab_a.find_unique_SentrixIDS()
    lab_b.find_unique_SentrixIDS()
    lab_c.find_unique_SentrixIDS()
    global_sentrix = serv.return_global_SentrixID(lab_a.unique_SentrixIDS,
                                lab_b.unique_SentrixIDS,
                                lab_c.unique_SentrixIDS)
    lab_a.SentrixID_effects(global_sentrix)
    lab_b.SentrixID_effects(global_sentrix)
    lab_c.SentrixID_effects(global_sentrix)
    



## Dasen normalisation

Client side

In [11]:
lab_a.intensity_distributions()
lab_b.intensity_distributions()
lab_c.intensity_distributions()

GSM3944749_6057833029_R04C02   -285.653249
GSM3944770_6057833029_R03C01   -319.071017
GSM3944795_6042324136_R01C01   -149.998474
GSM3944904_6055432167_R04C01   -419.324320
GSM3944972_6057833008_R03C01   -482.039857
GSM3945043_6057833013_R04C01   -448.469497
GSM3945118_6264488087_R01C01   -215.002899
GSM3945136_6042324060_R05C02   -695.669424
GSM3945152_6057833019_R06C01   -355.235450
GSM3945156_6057833010_R01C02   -142.063662
GSM3945263_6057825168_R01C02   -307.016205
GSM3945278_6057833018_R01C01   -317.850276
GSM3945394_6042324134_R04C02   -451.368755
dtype: float64


/home/silke/Documents/Fed_EWAS/Federated_Differential_Methylation_Analysis/Federated_Implementation/client.py:66: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  x[probe_type.squeeze() == "I"] = x[probe_type.squeeze() == "I"] - tI_correction


GSM3944749_6057833029_R04C02   -309.457686
GSM3944770_6057833029_R03C01   -276.955473
GSM3944795_6042324136_R01C01   -165.715506
GSM3944904_6055432167_R04C01   -144.505142
GSM3944972_6057833008_R03C01   -157.628101
GSM3945043_6057833013_R04C01   -127.109592
GSM3945118_6264488087_R01C01   -161.137730
GSM3945136_6042324060_R05C02   -230.109561
GSM3945152_6057833019_R06C01   -287.484359
GSM3945156_6057833010_R01C02   -194.250313
GSM3945263_6057825168_R01C02   -231.940672
GSM3945278_6057833018_R01C01   -146.488845
GSM3945394_6042324134_R04C02   -201.574755
dtype: float64
GSM3944820_6057833015_R01C01   -262.001404
GSM3944874_6057825109_R03C02   -366.069521
GSM3944876_6057833040_R03C02   -616.473891
GSM3944943_6164647017_R06C01   -622.730186
GSM3945008_6042324160_R04C01   -307.473983
GSM3945049_6057833036_R06C01   -520.035401
GSM3945060_6057833038_R06C02   -507.217627
GSM3945101_6057833018_R05C02   -491.348003
GSM3945250_6057833007_R05C01   -404.675436
GSM3945307_6055432164_R06C02   -479.445

In [12]:
local_dasen_paramA = lab_a.local_normalisation_parameters()
local_dasen_paramB = lab_b.local_normalisation_parameters()
local_dasen_paramC = lab_c.local_normalisation_parameters()

Server side

In [13]:
probe_type_means = serv.aggregate_QN_means(local_dasen_paramA, local_dasen_paramB, local_dasen_paramC)

Client side

In [14]:
lab_a.final_normalisation(probe_type_means)
lab_b.final_normalisation(probe_type_means)
lab_c.final_normalisation(probe_type_means)

/home/silke/Documents/Fed_EWAS/Federated_Differential_Methylation_Analysis/Federated_Implementation/client.py:309: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_out[0].set_index(self.methylated_dist[self.probe_annotation['Infinium_Design_Type'] =="I"].index, inplace = True)
/home/silke/Documents/Fed_EWAS/Federated_Differential_Methylation_Analysis/Federated_Implementation/client.py:312: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_out[1].set_index(self.methylated_dist[self.probe_annotation['Infinium_Design_Type'] =="II"].index, inplace = True)
/home/silke/Documents/Fed_EWAS/Federated_Differential_Methylation_Analysis/Federated_Implementation/client.py:318: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_out[2].set_index(self.methylated_dist[self.probe_annotation['Infinium_Design_Type'] =="I"].index, inplace = True)
/home/silke/Documents/Fed_EWAS/Federated_Differential_Methylati

In [15]:
# save the betas for testing
# lab_a.betas.to_csv(os.path.join(output, "strong_split1_betas.csv"))
# lab_b.betas.to_csv(os.path.join(output, "strong_split2_betas.csv"))
# lab_c.betas.to_csv(os.path.join(output, "strong_split3_betas.csv"))

## EWAS - Linear regression model

Client side

In [16]:
lab_a.local_xtx_xty()
lab_b.local_xtx_xty()
lab_c.local_xtx_xty()

  0%|          | 0/442337 [00:00<?, ?it/s]

OverflowError: cannot serialize a bytes object larger than 4 GiB

Server side

In [18]:
serv.global_regression_parameter((lab_a.xtx, lab_a.xty), (lab_b.xtx, lab_b.xty), (lab_c.xtx, lab_c.xty))

#### Client side

calculate the local sse and covariance of the regression coefficients

In [19]:
SSE_a,cov_coef_a = lab_a.compute_SSE_and_cov_coef(serv.beta)
SSE_b,cov_coef_b = lab_b.compute_SSE_and_cov_coef(serv.beta)
SSE_c,cov_coef_c = lab_c.compute_SSE_and_cov_coef(serv.beta)
SSE_list = [SSE_a, SSE_b, SSE_c]
cov_coef_list = [cov_coef_a, cov_coef_b, cov_coef_c]

#### Server side

calculate the global SSE and covariance of the regression coefficients

In [20]:
serv.aggregate_SSE_and_cov_coef(SSE_list,cov_coef_list)

In [21]:
np.savetxt(os.path.join(output, "strong_splits_model_matrix.csv"), serv.beta, delimiter=",") 
np.savetxt(os.path.join(output, "strong_splits_model_matrix_xty.csv"), serv.global_xty, delimiter=",") 

Make and fit the contrasts to the linear model

In [22]:
contrasts_mat = serv.make_contrasts(contrasts=[(["AD"],["CTRL"])])
serv.fit_contasts(contrasts_mat.values)

In [23]:
contrasts_mat.to_csv(os.path.join(output, "strong_splits_contrastmat.csv"))
np.savetxt(os.path.join(output, "strong_splits_model_matrix_contractfit.csv"), serv.beta, delimiter=",") 
np.savetxt(os.path.join(output, "strong_splits_model_matrix_xty_contractfit.csv"), serv.global_xty, delimiter=",") 

Calculate the P-values

In [24]:
serv.eBayes()

Get the results table

In [25]:
serv.table 

,t,lods,logFC,CI.L,CI.R,adj.P.Val,P.Value
cg11758688,-6.176943,9.331129,-2.664268e-01,-0.351587,-0.181266,0.002203,4.887204e-09
cg26516741,5.730027,7.130199,1.355506e-02,0.008884,0.018226,0.010446,4.633855e-08
cg27353962,5.595971,6.491377,3.453242e-02,0.022349,0.046716,0.013400,8.916686e-08
cg23289794,-5.469382,5.897633,-2.185569e-02,-0.029745,-0.013966,0.018480,1.639614e-07
cg09324326,5.412534,5.634057,7.487043e-02,0.047559,0.102182,0.019379,2.149250e-07
...,...,...,...,...,...,...,...
cg17413194,-0.000012,-7.915247,-9.141305e-08,-0.015253,0.015253,0.999998,9.999906e-01
cg22231902,0.000010,-7.915247,4.816494e-08,-0.009228,0.009228,0.999998,9.999918e-01
cg14481208,-0.000008,-7.915247,-9.116552e-08,-0.022621,0.022621,0.999998,9.999937e-01
cg03428314,0.000003,-7.915247,3.854627e-08,-0.024498,0.024498,0.999999,9.999975e-01


In [26]:
serv.table.to_csv(os.path.join(output, "strong_splits_EWAS_results.csv"))